In [36]:
# import the opencv library
import cv2

In [37]:
# import our model
from keras.models import load_model

In [38]:
# Other dependencies
import numpy as np
import tensorflow as tf
import os
from PIL import Image

In [39]:
# Crop center of the image feed
def crop_center(pil_img, crop_width, crop_height):
    img_width, img_height = pil_img.size
    return pil_img.crop(((img_width - crop_width) // 2,
                         (img_height - crop_height) // 2,
                         (img_width + crop_width) // 2,
                         (img_height + crop_height) // 2))

# Crop largest square from the image feed
def crop_max_square(pil_img):
    return crop_center(pil_img, min(pil_img.size), min(pil_img.size))

In [40]:
# define a video capture object
stream = cv2.VideoCapture(0)
model = load_model('emotion_model.h5')

# Creating text overlay
res,frame=stream.read()
height, width , channel = frame.shape
# Creating an Overlay window to write prediction and cofidence
sub_img = frame[0:int(height/6),0:int(width)]
black_rect = np.ones(sub_img.shape, dtype=np.uint8)*0
res = cv2.addWeighted(sub_img, 0.77, black_rect,0.23, 0)

FONT = cv2.FONT_HERSHEY_SIMPLEX
FONT_SCALE = 0.8
FONT_THICKNESS = 2
label_color = (10, 10, 255)
label = "Emotion Detection made by Group 4"
label_dimension = cv2.getTextSize(label,FONT ,FONT_SCALE,FONT_THICKNESS)[0]
textX = int((res.shape[1] - label_dimension[0]) / 2)
textY = int((res.shape[0] + label_dimension[1]) / 2)
cv2.putText(res, label, (textX,textY), FONT, FONT_SCALE, (0,0,0), FONT_THICKNESS)

while(True):
      
    # Capture the video frame by frame
    ret, frame = stream.read()
    
    #Convert the captured frame into RGB
    im = Image.fromarray(frame, 'RGB')
    
    #Resizing into dimensions you used while training
    im_new = crop_max_square(im)
    im_new = im_new.resize((48,48))
    img_array = np.array(im_new)
    
    #Expand dimensions to match the 4D Tensor shape.
    img_array = np.expand_dims(img_array, axis=1)
    img_array = img_array.reshape(-1, 48, 48, 1)
    img_array = img_array/255
        
    #Calling the predict function using keras
    prediction = model.predict(img_array)
    print(prediction)
    
    # Taking only the first of three arrays
    array = prediction[0]
    array_list = [float(x) for x in np.split(array, indices_or_sections=7)]
    array_list = np.asarray(array_list)
    
    predict = np.where(array_list == max(array_list))
    emotion = predict[0]
    
    # Part of the code that predicts the emotion from the prediction array
    if(emotion == 0):
        label = "Angry"
    elif(emotion == 1):
        label = "Disgust"
    elif(emotion == 2):
        label = "Fear"
    elif(emotion == 3):
        label = "Happy"
    elif(emotion == 4):
        label = "Sad"
    elif(emotion == 5):
        label = "Surprise"
    elif(emotion == 6):
        label = "Neutral"
    else:
        label = "Error"
            
    # Display the resulting frame
    cv2.imshow('frame', frame)
    cv2.putText(res, f"Emotion: + {label}", (0,textY+22+5), FONT,0.7, label_color,2)
    label_violation = f"Confidence: + {max(array_list)} + %"
    violation_text_dimension = cv2.getTextSize(label_violation,FONT,FONT_SCALE,FONT_THICKNESS )[0]
    violation_x_axis = int(res.shape[1]- violation_text_dimension[0])
    cv2.putText(res, label_violation, (violation_x_axis,textY+22+5), FONT, 0.7, label_color,2)
    
    # the 'q' button is set as the
    # quitting button you may use any
    # desired button of your choice
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break
  
# After the loop release the cap object
stream.release()

# Destroy all the windows
cv2.destroyAllWindows()

1/1 [==============================] - 0s 104ms/step
[[5.1719856e-01 1.1592859e-04 2.9066061e-03 3.1854108e-01 1.2675732e-01
  2.1159908e-04 3.4269016e-02]
 [6.6883294e-03 2.6279071e-04 1.7839670e-04 7.7002215e-01 1.5687664e-01
  4.0810917e-02 2.5160788e-02]
 [3.6557261e-03 5.8073417e-04 2.0832739e-04 7.2933042e-01 1.8865447e-01
  4.1140229e-02 3.6430072e-02]]
1/1 [==============================] - 0s 25ms/step
[[5.5047005e-01 1.1098695e-04 3.1224326e-03 2.7985686e-01 1.3259299e-01
  2.0136719e-04 3.3645242e-02]
 [7.9200445e-03 1.8696841e-04 1.1814406e-04 8.3049858e-01 9.5591597e-02
  2.4110435e-02 4.1574229e-02]
 [3.1887009e-04 4.7624347e-04 2.7363767e-05 7.8671628e-01 1.4802928e-01
  3.7573356e-02 2.6858635e-02]]
1/1 [==============================] - 0s 22ms/step
[[5.3927428e-01 1.2356030e-04 2.9761705e-03 2.8639597e-01 1.3672617e-01
  2.1191285e-04 3.4292001e-02]
 [6.1004921e-03 1.3087399e-04 8.6809931e-05 8.5799962e-01 7.5098097e-02
  2.1391824e-02 3.9192285e-02]
 [4.8456970e-04 4